In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [6]:
# Load the CSV file
file_path = r'C:\Users\obengamoako.n\Downloads\stations.csv'
stations_df = pd.read_csv(file_path, skiprows=1, header=0,
                          usecols=['Number', 'NAME', 'Lat', 'Long',
                                   'Municipality', 'Total Docks'],)

stations_df.head()

,Number,NAME,Lat,Long,Municipality,Total Docks
0,L32001,Railroad Lot and Minuteman Bikeway,42.416065,-71.153366,Arlington,11.0
1,L32002,Linwood St at Minuteman Bikeway,42.409354,-71.149065,Arlington,11.0
2,L32005,Thorndike Field at Minuteman Bikeway,42.400168,-71.144570,Arlington,11.0
3,L32003,Mass Ave at Grafton St,42.407261,-71.143821,Arlington,11.0
4,L32004,Broadway at Grafton St,42.409942,-71.140093,Arlington,11.0


In [7]:
# Create GeoDataFrame with station locations
stations_df['geometry'] = stations_df.apply(lambda row: Point(row['Long'], row['Lat']), axis=1)
stations_gdf = gpd.GeoDataFrame(stations_df, geometry='geometry', crs="EPSG:4326")

# Define area of interest around stations
convex_hull = stations_gdf.unary_union.convex_hull.buffer(0.02)

# Get road network from OpenStreetMap
G = ox.graph_from_polygon(convex_hull, network_type='drive')

# Project graph to UTM for distance calculations
G_proj = ox.project_graph(G)

# Project stations to the same CRS as the graph
stations_gdf = stations_gdf.to_crs(G_proj.graph['crs'])

stations_gdf.head()

,Number,NAME,Lat,Long,Municipality,Total Docks,geometry
0,L32001,Railroad Lot and Minuteman Bikeway,42.416065,-71.153366,Arlington,11.0,POINT (322825.928 4698219.344)
1,L32002,Linwood St at Minuteman Bikeway,42.409354,-71.149065,Arlington,11.0,POINT (323160.974 4697465.203)
2,L32005,Thorndike Field at Minuteman Bikeway,42.400168,-71.144570,Arlington,11.0,POINT (323505.154 4696435.812)
3,L32003,Mass Ave at Grafton St,42.407261,-71.143821,Arlington,11.0,POINT (323586.623 4697221.832)
4,L32004,Broadway at Grafton St,42.409942,-71.140093,Arlington,11.0,POINT (323900.925 4697511.870)


In [8]:
# Function to calculate a distance matrix
def calculate_distance_matrix(graph, stations_gdf):
    # Get station numbers and coordinates
    station_numbers = stations_gdf['Number'].tolist()
    station_coords = list(zip(stations_gdf.geometry.y, stations_gdf.geometry.x))

    # Initialize an empty DataFrame for the matrix
    distance_matrix = pd.DataFrame(index=station_numbers, columns=station_numbers, dtype=float)

    # Iterate through each pair of stations
    for i, (number1, coord1) in enumerate(zip(station_numbers, station_coords)):
        for j, (number2, coord2) in enumerate(zip(station_numbers, station_coords)):
            if number1 != number2:  # Avoid self-loops
                # Find nearest nodes on the graph
                orig_node = ox.distance.nearest_nodes(graph, coord1[1], coord1[0])
                dest_node = ox.distance.nearest_nodes(graph, coord2[1], coord2[0])

                # Check if a path exists between the nodes
                if nx.has_path(graph, orig_node, dest_node):
                    # Calculate shortest path length
                    path_length = nx.shortest_path_length(graph, orig_node, dest_node, weight='length')
                    distance_matrix.at[number1, number2] = path_length
                else:
                    # No path available
                    distance_matrix.at[number1, number2] = None

    return distance_matrix

# Calculate the distance matrix
distance_matrix = calculate_distance_matrix(G_proj, stations_gdf)

# Display the distance matrix
distance_matrix

,L32001,L32002,L32005,L32003,L32004,L32006,A32000,A32001,A32002,A32003,...,W32006,W32007,F32009,M32092,TBD - upcoming install,TBD - upcoming install,TBD - upcoming install,TBD - upcoming install,TBD - upcoming install,TBD - upcoming install
L32001,NaN,1269.466,2279.787,1469.433,1557.933,494.204,12288.899,9296.165,9794.830,9310.272,...,7536.665,6491.796,5432.271,8905.377,10098.741,10098.741,10098.741,10098.741,10098.741,10098.741
L32002,1119.835,NaN,1181.256,520.287,913.384,789.754,11339.753,8347.019,8845.684,8361.126,...,7552.411,6052.499,5261.925,7956.231,9321.769,9321.769,9321.769,9321.769,9321.769,9321.769
L32005,2130.156,1181.256,NaN,855.130,1302.636,1800.075,10758.851,7766.117,8264.782,7780.224,...,7537.611,5938.313,5867.970,7375.329,8740.867,8740.867,8740.867,8740.867,8740.867,8740.867
L32003,1319.802,520.287,857.504,NaN,458.030,989.721,10819.466,7826.732,8325.397,7840.839,...,7472.817,5972.905,5180.685,7435.944,8801.482,8801.482,8801.482,8801.482,8801.482,8801.482
L32004,1336.731,918.502,1412.527,555.023,NaN,1006.650,10913.793,8139.436,8638.101,8153.543,...,8027.840,6407.942,4771.616,7530.271,9114.186,9114.186,9114.186,9114.186,9114.186,9114.186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TBD - upcoming install,10165.346,9387.306,8806.404,8867.019,9239.719,9856.740,8911.186,1175.765,2282.168,3020.269,...,4054.055,3772.007,11682.424,7192.838,NaN,NaN,NaN,NaN,NaN,NaN
TBD - upcoming install,10165.346,9387.306,8806.404,8867.019,9239.719,9856.740,8911.186,1175.765,2282.168,3020.269,...,4054.055,3772.007,11682.424,7192.838,NaN,NaN,NaN,NaN,NaN,NaN
TBD - upcoming install,10165.346,9387.306,8806.404,8867.019,9239.719,9856.740,8911.186,1175.765,2282.168,3020.269,...,4054.055,3772.007,11682.424,7192.838,NaN,NaN,NaN,NaN,NaN,NaN
TBD - upcoming install,10165.346,9387.306,8806.404,8867.019,9239.719,9856.740,8911.186,1175.765,2282.168,3020.269,...,4054.055,3772.007,11682.424,7192.838,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Save the matrix to a CSV file
distance_matrix.to_csv(r'C:\Users\obengamoako.n\Downloads\station_distance_matrix.csv')

In [ ]:
# Lat/long coordinates for the start and end stations
start_lat, start_lng = 42.360167, -71.095005
end_lat, end_lng = 42.339288, -71.080330

# Function to plot route for given network type
def plot_route_for_network_type(graph, network_type, ax, title):
    # Get nearest nodes for the start and end locations
    start_node = ox.distance.nearest_nodes(graph, X=start_lng, Y=start_lat)
    end_node = ox.distance.nearest_nodes(graph, X=end_lng, Y=end_lat)

    # Calculate the shortest path based on the selected network type
    route = nx.shortest_path(graph, start_node, end_node, weight="length")  # "length" in meters

    # Get the route as a GeoDataFrame
    route_gdf = ox.routing.route_to_gdf(graph, route)

    # Calculate the total route length in meters (sum of the edge lengths)
    route_length_meters = route_gdf['length'].sum()

    # Convert route length to miles (1 mile = 1609.34 meters)
    route_length_miles = route_length_meters / 1609.34

    # Plotting the route
    ox.plot_graph_route(
        graph,
        route,
        route_linewidth=4,
        route_color="blue",
        node_size=0,
        bgcolor="white",
        ax=ax,
        show=False,
        close=False
    )

    # Zoom into the route area by setting plot bounds
    x_coords = [graph.nodes[node]['x'] for node in route]
    y_coords = [graph.nodes[node]['y'] for node in route]
    margin = 0.005
    ax.set_xlim(min(x_coords) - margin, max(x_coords) + margin)
    ax.set_ylim(min(y_coords) - margin, max(y_coords) + margin)

    # Add a basemap from contextily
    ctx.add_basemap(ax, crs=graph.graph['crs'], source=ctx.providers.CartoDB.Positron)

    # Add the route length (in miles) to the title
    ax.set_title(f"{title} - Distance: {route_length_miles:.2f} miles")

# Get the road network for Boston
place_name = "Boston, Massachusetts"
graph_bike = ox.graph_from_place(place_name, network_type="bike")
graph_all = ox.graph_from_place(place_name, network_type="all")

# Create a 1x2 grid of subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Plot bike route in the first subplot
plot_route_for_network_type(graph_bike, "bike", axes[0], "Bike Network Route")

# Plot all routes in the second subplot
plot_route_for_network_type(graph_all, "all", axes[1], "All Network Routes")

plt.tight_layout()
plt.show()

In [ ]:
# Define the bounding box with an offset
offset = 0.01
north = stations_df['Lat'].max() + offset
south = stations_df['Lat'].min() - offset
east = stations_df['Long'].max() + offset
west = stations_df['Long'].min() - offset

# Download the road network within the bounding box
G = ox.graph_from_bbox(north, south, east, west, network_type='drive')

# Find the nearest network node for each station
station_nodes = []
for _, row in stations_df.iterrows():
    nearest_node = ox.distance.nearest_nodes(G, row['Long'], row['Lat'])
    station_nodes.append(nearest_node)

# Create an empty DataFrame for the distance matrix
distance_matrix = pd.DataFrame(index=stations_df['Number'], columns=stations_df['Number'], dtype=float)

# Calculate shortest path distances between all pairs of stations
for i in range(len(station_nodes)):
    for j in range(i, len(station_nodes)):
        if i == j:
            distance_matrix.iloc[i, j] = 0  # Distance to itself is 0
        else:
            # Compute the shortest path distance in meters
            try:
                distance = nx.shortest_path_length(G, station_nodes[i], station_nodes[j], weight='length')
                distance_matrix.iloc[i, j] = distance / 1000  # Convert to kilometers
                distance_matrix.iloc[j, i] = distance / 1000  # Symmetric matrix
            except nx.NetworkXNoPath:
                distance_matrix.iloc[i, j] = None  # No path found
                distance_matrix.iloc[j, i] = None

print("Distance Matrix (in km):")
distance_matrix